# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [14]:
import napari
import os, glob
from macrohet import dataio #, visualise, notify
import numpy as np
import os
import re
import numpy as np
import cv2
import btrack
import zarr


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
%%time
expt_ID = 'ND0002'

base_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'

metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 33.1 s, sys: 2.92 s, total: 36 s
Wall time: 36 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0103K1F1P1R1,Ok,r01c03f01p01-ch1sk1fk1fl1.tiff,1,3,1,1,0,1,1,...,-2E-06,0.135466397,0,2023-11-30T17:22:09.49+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
1,0103K1F1P1R2,Ok,r01c03f01p01-ch2sk1fk1fl1.tiff,1,3,1,1,0,2,1,...,-2E-06,0.135466397,0,2023-11-30T17:22:09.723+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
2,0103K1F1P2R1,Ok,r01c03f01p02-ch1sk1fk1fl1.tiff,1,3,1,2,0,1,1,...,0,0.135468394,0,2023-11-30T17:22:10.067+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
3,0103K1F1P2R2,Ok,r01c03f01p02-ch2sk1fk1fl1.tiff,1,3,1,2,0,2,1,...,0,0.135468394,0,2023-11-30T17:22:10.287+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
4,0103K1F1P3R1,Ok,r01c03f01p03-ch1sk1fk1fl1.tiff,1,3,1,3,0,1,1,...,2E-06,0.135470405,0,2023-11-30T17:22:10.627+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388615,0612K150F9P1R2,Ok,r06c12f09p01-ch2sk150fk1fl1.tiff,6,12,9,1,149,2,1,...,-2E-06,0.1351538,268191.66,2023-12-03T20:06:16.08+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388616,0612K150F9P2R1,Ok,r06c12f09p02-ch1sk150fk1fl1.tiff,6,12,9,2,149,1,1,...,0,0.135155797,268191.66,2023-12-03T20:06:16.423+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388617,0612K150F9P2R2,Ok,r06c12f09p02-ch2sk150fk1fl1.tiff,6,12,9,2,149,2,1,...,0,0.135155797,268191.66,2023-12-03T20:06:16.657+00:00,488,522,40,1.1,0.1,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."
388618,0612K150F9P3R1,Ok,r06c12f09p03-ch1sk150fk1fl1.tiff,6,12,9,3,149,1,1,...,2E-06,0.135157794,268191.66,2023-12-03T20:06:17+00:00,640,706,40,1.1,0.2,"[[1.000989,0,0,10.0],[0,-1.000989,0,-6.8],[0,0..."


In [15]:
base_dir = f'/mnt/SYNO/leo/Monocyte-Infect-T24hr_241007__2024-10-07T16_51_32-Measurement 1/'

metadata_fn = glob.glob(os.path.join(base_dir, 'Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0101K1F1P1R1,Ok,r01c01f01p01-ch1sk1fk1fl1.tiff,1,1,1,1,0,1,1,...,-2E-06,0.135289997,0,2024-10-07T17:11:45.74+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
1,0101K1F1P1R2,Ok,r01c01f01p01-ch2sk1fk1fl1.tiff,1,1,1,1,0,2,1,...,-2E-06,0.135289997,0,2024-10-07T17:11:45.757+01:00,640,706,40,1.1,0.2,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
2,0101K1F1P1R3,Ok,r01c01f01p01-ch3sk1fk1fl1.tiff,1,1,1,1,0,3,1,...,-2E-06,0.135289997,0,2024-10-07T17:11:46.087+01:00,488,522,40,1.1,0.16,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
3,0101K1F1P1R4,Ok,r01c01f01p01-ch4sk1fk1fl1.tiff,1,1,1,1,0,4,1,...,-2E-06,0.135289997,0,2024-10-07T17:11:46.1+01:00,561,599,40,1.1,0.2,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
4,0101K1F1P2R1,Ok,r01c01f01p02-ch1sk1fk1fl1.tiff,1,1,1,2,0,1,1,...,-1E-06,0.135290995,0,2024-10-07T17:11:46.443+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24763,0703K1F43P5R4,Ok,r07c03f43p05-ch4sk1fk1fl1.tiff,7,3,43,5,0,4,1,...,2E-06,0.135141402,0,2024-10-07T17:41:36.483+01:00,561,599,40,1.1,0.2,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
24764,0703K1F43P6R1,Ok,r07c03f43p06-ch1sk1fk1fl1.tiff,7,3,43,6,0,1,1,...,3E-06,0.135142401,0,2024-10-07T17:41:36.827+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
24765,0703K1F43P6R2,Ok,r07c03f43p06-ch2sk1fk1fl1.tiff,7,3,43,6,0,2,1,...,3E-06,0.135142401,0,2024-10-07T17:41:36.827+01:00,640,706,40,1.1,0.2,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
24766,0703K1F43P6R3,Ok,r07c03f43p06-ch3sk1fk1fl1.tiff,7,3,43,6,0,3,1,...,3E-06,0.135142401,0,2024-10-07T17:41:37.17+01:00,488,522,40,1.1,0.16,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."


In [25]:
len(metadata[['Row', 'Col']].drop_duplicates())

24

In [26]:
metadata['TimepointID'].unique()

array(['0'], dtype=object)

In [21]:
metadata['FieldID'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43'], dtype=object)

In [28]:
metadata[(metadata['FieldID'] == '1') & (metadata['ChannelID'] == '1') & (metadata['PositionZ'] == '-2E-06')]

,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0101K1F1P1R1,Ok,r01c01f01p01-ch1sk1fk1fl1.tiff,1,1,1,1,0,1,1,...,-2E-06,0.135289997,0,2024-10-07T17:11:45.74+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
1032,0102K1F1P1R1,Ok,r01c02f01p01-ch1sk1fk1fl1.tiff,1,2,1,1,0,1,1,...,-2E-06,0.135262504,0,2024-10-07T17:15:03.003+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
2064,0103K1F1P1R1,Ok,r01c03f01p01-ch1sk1fk1fl1.tiff,1,3,1,1,0,1,1,...,-2E-06,0.135232702,0,2024-10-07T17:58:18.613+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
3096,0201K1F1P1R1,Ok,r02c01f01p01-ch1sk1fk1fl1.tiff,2,1,1,1,0,1,1,...,-2E-06,0.135286793,0,2024-10-07T17:08:27.59+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
4128,0202K1F1P1R1,Ok,r02c02f01p01-ch1sk1fk1fl1.tiff,2,2,1,1,0,1,1,...,-2E-06,0.135249704,0,2024-10-07T17:18:23.043+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
5160,0203K1F1P1R1,Ok,r02c03f01p01-ch1sk1fk1fl1.tiff,2,3,1,1,0,1,1,...,-2E-06,0.135211304,0,2024-10-07T17:54:59.2+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
6192,0301K1F1P1R1,Ok,r03c01f01p01-ch1sk1fk1fl1.tiff,3,1,1,1,0,1,1,...,-2E-06,0.135279402,0,2024-10-07T17:05:08.86+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
7224,0302K1F1P1R1,Ok,r03c02f01p01-ch1sk1fk1fl1.tiff,3,2,1,1,0,1,1,...,-2E-06,0.135233998,0,2024-10-07T17:21:40.913+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
8256,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,-2E-06,0.135187998,0,2024-10-07T17:51:38.613+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."
9288,0401K1F1P1R1,Ok,r04c01f01p01-ch1sk1fk1fl1.tiff,4,1,1,1,0,1,1,...,-2E-06,0.135262296,0,2024-10-07T17:01:48.853+01:00,405,456,40,1.1,0.1,"[[0.987277,0,0,-7.5],[0,-0.987277,0,-0.8],[0,0..."


In [32]:
from skimage import io

napari.Viewer().add_image(io.imread(os.path.join(base_dir, 'Images/', 'r01c01f01p01-ch1sk1fk1fl1.tiff')))

<Image layer 'Image' at 0x7f533edee860>

### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [16]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,replicate_number=False)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

IndexError: list index out of range

### Load using Zarr

In [4]:
acq_ID = (4, 4)

In [5]:
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')

zarr_group = zarr.open(image_dir, mode='r')

In [6]:
%%time
images = zarr_group.images[:,1,1,...]
images.shape

CPU times: user 9.75 s, sys: 4.12 s, total: 13.9 s
Wall time: 3.67 s


(150, 6048, 6048)

In [ ]:
images_max_proj = np.max(zarr_group.images, axis = 2)

In [8]:
images_max_proj.shape

(154, 2, 6048, 6048)

In [7]:
images.shape

(150, 6048, 6048)

#### Load downscaled version from mp4 videos

In [8]:
mp4_path = glob.glob(f'/mnt/SYNO/videos/macrohet_videos/{expt_ID}/{expt_ID}_{acq_ID}*')[0]

# Launch napari

In [8]:
viewer = napari.Viewer(title = f'{expt_ID, acq_ID}, max_proj')


viewer.add_image(images, #_max_proj,
                 # channel_axis = 1, 
                 # colormap=['magenta', 'green'],
                 blending = 'additive', 
                 # contrast_limits=[[350, 1000],[0, 7000]]
                )

<Image layer 'images' at 0x7f55ea6ef460>

In [9]:
with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/cpv3/{acq_ID}.h5'), #macrohet_seg_model 
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                segmentation = reader.segmentation
                tracks = reader.tracks

[INFO][2024/10/02 04:20:52 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0002/labels/cpv3/(4, 4).h5...
[INFO][2024/10/02 04:21:12 pm] Loading segmentation (150, 6048, 6048)
[INFO][2024/10/02 04:21:12 pm] Loading tracks/obj_type_1
[INFO][2024/10/02 04:21:12 pm] Loading LBEP/obj_type_1
[INFO][2024/10/02 04:21:12 pm] Loading objects/obj_type_1 (27406, 5) (27406 filtered: None)
[INFO][2024/10/02 04:21:12 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0002/labels/cpv3/(4, 4).h5


### Add tracks

In [10]:
filtered_tracks = [track for track in tracks if len(track) > 35]

In [27]:
recoloured_seg = btrack.utils.update_segmentation(segmentation, filtered_tracks, scale = (5.04, 5.04))

In [11]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(filtered_tracks, ndim = 2)

In [12]:
viewer.add_tracks(napari_tracks, scale=(1, 5.04, 5.04), name = 'filtered')

<Tracks layer 'filtered' at 0x7f530c8426b0>

### Add segmentation

In [13]:
viewer.add_labels(segmentation)
# viewer.add_labels(recoloured_seg)

<Labels layer 'segmentation' at 0x7f530c30d750>

/home/dayn/miniconda3/envs/godspeed/lib/python3.10/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
INFO:trackastra.model.model:Loading model state from /home/dayn/.trackastra/.models/general_2d/model.pt


/home/dayn/.trackastra/.models/general_2d already downloaded, skipping.


/home/dayn/miniconda3/envs/godspeed/lib/python3.10/site-packages/trackastra/model/model.py:498: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(fpath, map_l

Tracking with mode greedy...


INFO:trackastra.data.wrfeat:Extracting features from 150 detections
INFO:trackastra.data.wrfeat:Using single process for feature extraction
INFO:trackastra.model.model_api:Building windows
INFO:trackastra.model.model_api:Predicting windows
INFO:trackastra.model.model_api:Running greedy tracker
INFO:trackastra.tracking.tracking:Build candidate graph with delta_t=1
INFO:trackastra.tracking.tracking:Added 33107 vertices, 24877 edges                                                    
INFO:trackastra.tracking.tracking:Running greedy tracker
Converting graph to CTC results: 100%|████████████████████████████████████████████| 4613/4613 [00:12<00:00, 380.21it/s]


In [28]:
viewer.add_image(images_max_proj[-5],
                 channel_axis = 0, 
                 colormap=['green', 'magenta'],
                 blending = 'additive', 
                 contrast_limits=[[0, 2400],[0, 1000]]
                )

[<Image layer 'Image [4]' at 0x7fa481516460>,
 <Image layer 'Image [5]' at 0x7fa4823087f0>]

In [ ]:
viewer.camera.reset()
`

In [14]:
visualise.highlight_cell(126, viewer, filtered_tracks, scale_factor=5.02)

<Points layer 'cell 126' at 0x7f248be4fa60>

In [20]:
for layer in viewer.layers:
    viewer.layers[layer.name].scale = (10, 1, 1)

In [21]:
viewer.layers[2].name

'filtered'

In [22]:
viewer.layers[2].scale = (10,5.04, 5.04)